In [24]:
from utils import dsl
from utils import constants
from utils.dsl import height, width
import json
import numpy as np
from utils.dsl import *

In [29]:
task_id = '7b80bb43'
def solve(I):
    # Identify connected components: single-color, 4-connected, ignore background
    x1 = objects(I, True, False, True)
    
    # Separate single-cell particles and larger blobs
    x2 = sizefilter(x1, 1)
    x3 = difference(x1, x2)
    
    # Get colors present in blobs
    x4 = apply(color, x3)
    
    # Function to check if a particle's color has a corresponding blob
    x5 = rbind(contained, x4)
    x6 = compose(x5, color)
    
    # Filter particles that can move (have a larger blob of same color)
    x7 = sfilter(x2, x6)
    
    # Create base grid by erasing moving particles only
    x8 = merge(x7)
    x9 = cover(I, x8)
    
    # Compute new positions for each moving particle: find nearest blob, gravitate, then shift
    new_positions = []
    for s in x7:
        c = color(s)
        same_blobs = colorfilter(x3, c)
        pos = ulcorner(toindices(s))
        # compute nearest blob by minimum Manhattan distance from s to any cell in blob
        def blob_min_distance(blob):
            blob_indices = toindices(blob)
            distances = apply(lbind(manhattan, pos), blob_indices)
            min_distance = valmin(distances, identity)
            return min_distance

        nearest = argmin(same_blobs, blob_min_distance)
        offset = gravitate(s, nearest)
        new_s = shift(s, offset)
        new_positions.append(new_s)
    
    # Merge all new particle positions into one object
    x10 = merge(new_positions)
    
    # Paint the new positions onto the base grid
    O = paint(x9, x10)
    return O

In [30]:
puzzle_index = 1
with open(f'../data_v2/evaluation/{task_id}.json') as f:
    task = json.load(f)
I = task['train'][puzzle_index]['input']
expected = task['train'][puzzle_index]['output']
I=tuple(tuple(row) for row in I)
output = solve(I)

TypeError: 'int' object is not subscriptable

In [20]:
for row in I:
    print(row)


(1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0)
(0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1)
(1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0)
(0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 4, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1)
(1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 3, 0, 4, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0)
(0, 1, 0, 1, 0, 1, 0, 2, 0, 3, 0, 3, 0, 4, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1)
(1, 0, 1, 0, 1, 0, 1, 0, 3, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0)
(0, 1, 0, 1, 0, 1, 0, 4, 0, 3, 0, 3, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1)
(1, 0, 1, 0, 1, 0, 1, 0, 4, 0, 3, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0)
(0, 1, 0, 1, 0, 1, 0, 1, 0, 4, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1)
(1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0)

In [21]:
for row in output:
    print(row)

(1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 4, 0, 1, 0, 4, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0)
(0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 4, 0, 1, 0, 4, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1)
(1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 4, 0, 1, 0, 4, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0)
(0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 4, 0, 1, 0, 4, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1)
(1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 3, 0, 4, 0, 4, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0)
(0, 1, 0, 1, 0, 1, 0, 2, 0, 3, 0, 3, 0, 4, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1)
(1, 0, 1, 0, 1, 0, 1, 0, 3, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0)
(0, 1, 0, 1, 0, 1, 0, 4, 0, 3, 0, 3, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1)
(1, 0, 1, 0, 1, 0, 4, 0, 4, 0, 3, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0)
(0, 1, 0, 1, 0, 4, 0, 1, 0, 4, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1)
(1, 0, 1, 0, 4, 0, 1, 0, 4, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0)

In [22]:
for row in expected:
    print(row)

[1, 0, 2, 0, 1, 0, 2, 0, 1, 0, 1, 0, 1, 0, 4, 0, 1, 0, 4, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
[0, 1, 0, 2, 0, 1, 0, 2, 0, 1, 0, 1, 0, 4, 0, 1, 0, 4, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
[1, 0, 1, 0, 2, 0, 1, 0, 2, 0, 1, 0, 4, 0, 1, 0, 4, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
[0, 1, 0, 1, 0, 2, 0, 1, 0, 2, 0, 4, 0, 1, 0, 4, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
[1, 0, 1, 0, 1, 0, 2, 0, 2, 0, 3, 0, 4, 0, 4, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
[0, 1, 0, 1, 0, 1, 0, 2, 0, 3, 0, 3, 0, 4, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
[1, 0, 1, 0, 1, 0, 1, 0, 3, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
[0, 1, 0, 1, 0, 1, 0, 4, 0, 3, 0, 3, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
[1, 0, 1, 0, 1, 0, 4, 0, 4, 0, 3, 0, 2, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
[0, 1, 0, 1, 0, 4, 0, 1, 0, 4, 0, 2, 0, 1, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
[1, 0, 1, 0, 4, 0, 1, 0, 4, 0, 1, 0, 2, 0, 1, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]

In [ ]:
expected_np = np.array(expected)
output_np = np.array(output)

if np.ndim(expected_np) != np.ndim(output_np) or expected_np.shape != output_np.shape:
    print("Dimension mismatch between expected and output.")
    print("expected shape:", expected_np.shape)

else:
    diffs = np.where(expected_np != output_np)
    # print differences as ascii art
    diff_grid = np.full(expected_np.shape, ' ')
    for r, c in zip(*diffs):
        diff_grid[r, c] = 'X'
    for row in diff_grid:
        print(''.join(row))  

  X   X                       
   X   X                      
    X   X                     
     X                        
      X                       
                              
                              
                              
              X               
               X              
            X   X             
             X   X            
              X   X           
               X   X          
                X   X         
                 X   X        
                  X   X       
                   X   X      
                    X   X     
                     X   X    
                      X   X   
                       X   X  
                        X   X 
                         X   X
